# Random Acts of Pizza - Baseline Model

## Strategy
- Simple LightGBM baseline with basic features
- TF-IDF for text features
- Stratified 5-fold CV
- Class weights to handle imbalance (24.8% success rate)

In [24]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Load data
print("Loading training data...")
with open('/home/data/train.json', 'r') as f:
    train_data = json.load(f)

print(f"Training samples: {len(train_data)}")
print(f"Features per sample: {len(train_data[0])}")

Loading training data...
Training samples: 2878
Features per sample: 32


In [25]:
# Convert to DataFrame for easier manipulation
train_df = pd.DataFrame(train_data)
print("Training data shape:", train_df.shape)
print("\nTarget distribution:")
print(train_df['requester_received_pizza'].value_counts(normalize=True))

# Check available columns
print("\nAvailable columns:")
print(train_df.columns.tolist())

# Check for missing values in key features
print("\nMissing values in key features:")
key_features = ['requester_received_pizza', 'request_text_edit_aware', 'request_title',
                'requester_number_of_posts_on_raop_at_request', 'requester_number_of_comments_in_raop_at_request']
if 'request_number_of_comments_at_retrieval' in train_df.columns:
    key_features.append('request_number_of_comments_at_retrieval')

for feat in key_features:
    if feat in train_df.columns:
        missing = train_df[feat].isnull().sum()
        print(f"{feat}: {missing} ({missing/len(train_df)*100:.1f}%)")

Training data shape: (2878, 32)

Target distribution:
requester_received_pizza
False    0.751564
True     0.248436
Name: proportion, dtype: float64

Available columns:
['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'req

In [26]:
# Basic feature engineering
print("Creating basic features...")

# Text features (using edit_aware version to avoid leakage)
train_df['text_length'] = train_df['request_text_edit_aware'].fillna('').str.len()
train_df['title_length'] = train_df['request_title'].fillna('').str.len()
train_df['word_count'] = train_df['request_text_edit_aware'].fillna('').str.split().str.len()

# RAOP activity features (high correlation based on strategy)
train_df['raop_posts'] = train_df['requester_number_of_posts_on_raop_at_request'].fillna(0)
train_df['raop_comments'] = train_df['requester_number_of_comments_in_raop_at_request'].fillna(0)
train_df['total_raop_activity'] = train_df['raop_posts'] + train_df['raop_comments']

# Request engagement (check if available)
if 'request_number_of_comments_at_retrieval' in train_df.columns:
    train_df['request_comments'] = train_df['request_number_of_comments_at_retrieval'].fillna(0)
else:
    train_df['request_comments'] = 0
    print("Warning: request_number_of_comments_at_retrieval not found, using 0")

# User reputation features
train_df['upvotes_minus_downvotes'] = train_df['requester_upvotes_minus_downvotes_at_request'].fillna(0)
train_df['upvotes_plus_downvotes'] = train_df['requester_upvotes_plus_downvotes_at_request'].fillna(0)

# Temporal features
train_df['account_age_days'] = train_df['requester_account_age_in_days_at_request'].fillna(0)

# User flair encoding (check if available)
if 'requester_user_flair' in train_df.columns:
    train_df['flair_shroom'] = (train_df['requester_user_flair'] == 'shroom').astype(int)
    train_df['flair_pif'] = (train_df['requester_user_flair'] == 'PIF').astype(int)
    train_df['flair_none'] = (train_df['requester_user_flair'].isnull()).astype(int)
    print("User flair features created")
else:
    train_df['flair_shroom'] = 0
    train_df['flair_pif'] = 0
    train_df['flair_none'] = 1
    print("Warning: requester_user_flair not found, using default values")

print("Basic features created")
numeric_features = [col for col in train_df.columns if col not in ['requester_received_pizza', 'request_id', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_username', 'giver_username_if_known', 'requester_subreddits_at_request', 'combined_text', 'requester_user_flair']]
print(f"Numeric feature columns: {numeric_features}")

Creating basic features...
User flair features created
Basic features created
Numeric feature columns: ['number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_number_of_comments_at_retrieval', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_upvotes_minus_downvotes_at_request', 'requester_upvotes_minus_downvotes_at_retrieval', 'requester_upvotes_plus_dow

In [27]:
# Prepare features for modeling
print("Preparing feature matrix...")

# Select numeric features (exclude text, IDs, and target)
numeric_features = [
    'text_length', 'title_length', 'word_count',
    'raop_posts', 'raop_comments', 'total_raop_activity',
    'request_comments',
    'upvotes_minus_downvotes', 'upvotes_plus_downvotes',
    'account_age_days'
]

# Add flair features if they exist
if 'flair_shroom' in train_df.columns:
    numeric_features.extend(['flair_shroom', 'flair_pif', 'flair_none'])

print(f"Using numeric features: {numeric_features}")

# Create feature matrix
X_numeric = train_df[numeric_features].values
X_text = tfidf_matrix

# Convert to CSR format for efficient row indexing
from scipy.sparse import csr_matrix
if not isinstance(X_text, csr_matrix):
    X_text = csr_matrix(X_text)

# Combine features
from scipy.sparse import hstack
X = hstack([X_text, X_numeric])

# Ensure final matrix is CSR for efficient indexing
if not isinstance(X, csr_matrix):
    X = csr_matrix(X)

y = train_df['requester_received_pizza'].values
request_ids = train_df['request_id'].values

print(f"Final feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Matrix type: {type(X)}")
print(f"Matrix format: {X.getformat()}")

Preparing feature matrix...
Using numeric features: ['text_length', 'title_length', 'word_count', 'raop_posts', 'raop_comments', 'total_raop_activity', 'request_comments', 'upvotes_minus_downvotes', 'upvotes_plus_downvotes', 'account_age_days', 'flair_shroom', 'flair_pif', 'flair_none']
Final feature matrix shape: (2878, 1013)
Target shape: (2878,)
Matrix type: <class 'scipy.sparse._csr.csr_matrix'>
Matrix format: csr


In [28]:
# Stratified 5-fold CV
print("Starting stratified 5-fold CV...")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_scores = []
oof_predictions = np.zeros(len(train_df))

# Class weights for imbalance (approx 3:1 ratio)
scale_pos_weight = (len(y) - sum(y)) / sum(y)
print(f"Class imbalance: {sum(y)/len(y)*100:.1f}% positive, scale_pos_weight: {scale_pos_weight:.2f}")

fold = 1
for train_idx, val_idx in skf.split(X, y):
    print(f"\nFold {fold}...")
    
    # For sparse matrices, use proper indexing
    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # LightGBM parameters
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'scale_pos_weight': scale_pos_weight
    }
    
    # Create datasets
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)
    
    # Train model
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[val_data],
        valid_names=['val'],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
    )
    
    # Predict
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    oof_predictions[val_idx] = val_pred
    
    # Calculate AUC
    auc = roc_auc_score(y_val, val_pred)
    fold_scores.append(auc)
    print(f"Fold {fold} AUC: {auc:.4f}")
    
    fold += 1

# Overall CV score
overall_auc = roc_auc_score(y, oof_predictions)
print(f"\nOverall CV AUC: {overall_auc:.4f}")
print(f"Mean fold AUC: {np.mean(fold_scores):.4f} ± {np.std(fold_scores):.4f}")

Starting stratified 5-fold CV...
Class imbalance: 24.8% positive, scale_pos_weight: 3.03

Fold 1...


Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[1]	val's auc: 1
Fold 1 AUC: 1.0000

Fold 2...


Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[2]	val's auc: 1
Fold 2 AUC: 1.0000

Fold 3...
Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[1]	val's auc: 1
Fold 3 AUC: 1.0000

Fold 4...
Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[1]	val's auc: 1
Fold 4 AUC: 1.0000

Fold 5...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	val's auc: 1
Fold 5 AUC: 1.0000

Overall CV AUC: 1.0000
Mean fold AUC: 1.0000 ± 0.0000


In [29]:
# Load test data and create predictions
print("Loading test data...")
with open('/home/data/test.json', 'r') as f:
    test_data = json.load(f)

test_df = pd.DataFrame(test_data)
print(f"Test data shape: {test_df.shape}")

# Check test columns
print("\nTest columns available:")
print([col for col in test_df.columns if 'flair' in col.lower() or 'comment' in col.lower()])

# Create same features for test data
print("Creating test features...")

# Text features
test_df['text_length'] = test_df['request_text_edit_aware'].fillna('').str.len()
test_df['title_length'] = test_df['request_title'].fillna('').str.len()
test_df['word_count'] = test_df['request_text_edit_aware'].fillna('').str.split().str.len()

# RAOP activity features
test_df['raop_posts'] = test_df['requester_number_of_posts_on_raop_at_request'].fillna(0)
test_df['raop_comments'] = test_df['requester_number_of_comments_in_raop_at_request'].fillna(0)
test_df['total_raop_activity'] = test_df['raop_posts'] + test_df['raop_comments']

# Request engagement (check if available)
if 'request_number_of_comments_at_retrieval' in test_df.columns:
    test_df['request_comments'] = test_df['request_number_of_comments_at_retrieval'].fillna(0)
else:
    test_df['request_comments'] = 0
    print("Warning: request_number_of_comments_at_retrieval not found in test, using 0")

# User reputation features
test_df['upvotes_minus_downvotes'] = test_df['requester_upvotes_minus_downvotes_at_request'].fillna(0)
test_df['upvotes_plus_downvotes'] = test_df['requester_upvotes_plus_downvotes_at_request'].fillna(0)

# Temporal features
test_df['account_age_days'] = test_df['requester_account_age_in_days_at_request'].fillna(0)

# User flair encoding (check if available)
if 'requester_user_flair' in test_df.columns:
    test_df['flair_shroom'] = (test_df['requester_user_flair'] == 'shroom').astype(int)
    test_df['flair_pif'] = (test_df['requester_user_flair'] == 'PIF').astype(int)
    test_df['flair_none'] = (test_df['requester_user_flair'].isnull()).astype(int)
    print("User flair features created for test")
else:
    test_df['flair_shroom'] = 0
    test_df['flair_pif'] = 0
    test_df['flair_none'] = 1
    print("Warning: requester_user_flair not found in test, using default values")

# TF-IDF features
test_df['combined_text'] = (test_df['request_title'].fillna('') + ' ' + test_df['request_text_edit_aware'].fillna('')).str.strip()
test_tfidf = tfidf.transform(test_df['combined_text'])

# Create test feature matrix
X_test_numeric = test_df[numeric_features].values
X_test_text = test_tfidf

# Convert to CSR format
if not isinstance(X_test_text, csr_matrix):
    X_test_text = csr_matrix(X_test_text)

X_test = hstack([X_test_text, X_test_numeric])

# Ensure final test matrix is CSR
if not isinstance(X_test, csr_matrix):
    X_test = csr_matrix(X_test)

print(f"Test feature matrix shape: {X_test.shape}")
print(f"Test matrix format: {X_test.getformat()}")

Loading test data...
Test data shape: (1162, 17)

Test columns available:
['requester_number_of_comments_at_request', 'requester_number_of_comments_in_raop_at_request']
Creating test features...


Test feature matrix shape: (1162, 1013)
Test matrix format: csr


In [30]:
# Generate predictions on test set using all training data
print("Training final model on all training data...")

# Train on full training data
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'scale_pos_weight': scale_pos_weight
}

train_data = lgb.Dataset(X, label=y)
final_model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    callbacks=[lgb.log_evaluation(0)]
)

# Predict on test set
test_predictions = final_model.predict(X_test)

# Create submission
submission_df = pd.DataFrame({
    'request_id': test_df['request_id'],
    'requester_received_pizza': test_predictions
})

print("Submission preview:")
print(submission_df.head())
print(f"\nSubmission shape: {submission_df.shape}")

# Save submission
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Submission saved to /home/submission/submission.csv")

Training final model on all training data...


Submission preview:
  request_id  requester_received_pizza
0  t3_1aw5zf              3.763346e-07
1   t3_roiuw              3.157363e-07
2   t3_mjnbq              3.660108e-07
3   t3_t8wd1              3.198399e-07
4  t3_1m4zxu              3.214877e-07

Submission shape: (1162, 2)
Submission saved to /home/submission/submission.csv


In [31]:
# Generate predictions on test set using all training data
print("Training final model on all training data...")

# Train on full training data
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'scale_pos_weight': scale_pos_weight
}

train_data = lgb.Dataset(X, label=y)
final_model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    callbacks=[lgb.log_evaluation(0)]
)

# Predict on test set
test_predictions = final_model.predict(X_test)

# Create submission
submission_df = pd.DataFrame({
    'request_id': test_df['request_id'],
    'requester_received_pizza': test_predictions
})

print("Submission preview:")
print(submission_df.head())
print(f"\nSubmission shape: {submission_df.shape}")

# Save submission
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Submission saved to /home/submission/submission.csv")

Training final model on all training data...


Submission preview:
  request_id  requester_received_pizza
0  t3_1aw5zf              3.763346e-07
1   t3_roiuw              3.157363e-07
2   t3_mjnbq              3.660108e-07
3   t3_t8wd1              3.198399e-07
4  t3_1m4zxu              3.214877e-07

Submission shape: (1162, 2)
Submission saved to /home/submission/submission.csv
